<a href="https://colab.research.google.com/github/Nithinps021/Project_IISU/blob/main/Chatbot_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from urllib import response
import numpy as np
import random
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Bidirectional, Dropout, Dense, Activation, Flatten, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import SGD

data_file = open('intents_cleaned_1.json').read()
intents = json.loads(data_file)
intents=intents['intents']
random.shuffle(intents)
classes=[]
alldata=[]
responses={}
for i in intents:
	if i['tag'] not in classes:
		classes.append(i['tag'])
		responses[i['tag']]=i['responses'][0]
	for j in i['patterns']:
		alldata.append([j,i['tag']])


random.shuffle(alldata)
tot_data=len(alldata)
tot_classes=len(classes)
x_train=np.array([])
y_train=np.zeros(shape=(tot_data,tot_classes), dtype=np.uint8)



In [ ]:
import spacy
nlp =spacy.load("en_core_web_sm")

def normalize(text):
  text=nlp(text)
  norm_text=[]
  for token in text:
    if not token.is_punct and not token.is_stop and not token.is_space:
      norm_text.append(token.lemma_.lower())
  return ' '.join(norm_text)

for i in range(tot_data):
  sen = normalize(alldata[i][0])
  print(sen)
  x_train=np.append(x_train,sen)
  y_train[i][classes.index(alldata[i][1])]=1


In [ ]:
text=x_train
Tokenizer = Tokenizer()
Tokenizer.fit_on_texts(text) 
Tokenizer_vocab_size = len(Tokenizer.word_index) + 1
Tokenizer_vocab_size, x_train.shape, y_train.shape


print(x_train)

x_val=x_train[0:157]
y_val=y_train[0:157]

x_train=x_train[157:]
y_train=y_train[157:]

X_train_encoded_words = Tokenizer.texts_to_sequences(x_train)
X_val_encoded_words = Tokenizer.texts_to_sequences(x_val)
X_train_encoded_padded_words = sequence.pad_sequences(X_train_encoded_words, maxlen =100)
X_val_encoded_padded_words = sequence.pad_sequences(X_val_encoded_words, maxlen = 100)

In [30]:

model = Sequential()

model.add(Embedding(Tokenizer_vocab_size,100, input_length = 100)) # Accepts the vector inputs
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64))) 
model.add(Dropout(0.5))
model.add(Dense(800, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(tot_classes, activation='softmax'))

model.summary()

Nadam = tf.keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
model.compile(loss='categorical_crossentropy', optimizer=Nadam, metrics=['accuracy'])
history  = model.fit(X_train_encoded_padded_words,y_train, epochs = 100, batch_size=100, verbose=1, validation_data=(X_val_encoded_padded_words, y_val))
model.save('gru3.h5')

print(len(x_train),y_train)
print(Tokenizer_vocab_size)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 100, 100)          93100     
                                                                 
 bidirectional_13 (Bidirecti  (None, 128)              63744     
 onal)                                                           
                                                                 
 dropout_36 (Dropout)        (None, 128)               0         
                                                                 
 dense_36 (Dense)            (None, 800)               103200    
                                                                 
 dropout_37 (Dropout)        (None, 800)               0         
                                                                 
 dense_37 (Dense)            (None, 200)               160200    
                                                     

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)


16/16 [==============================] - 9s 281ms/step - loss: 5.8300 - accuracy: 0.0072 - val_loss: 5.8320 - val_accuracy: 0.0255
Epoch 2/100
16/16 [==============================] - 4s 224ms/step - loss: 5.8133 - accuracy: 0.0118 - val_loss: 5.8461 - val_accuracy: 0.0255
Epoch 3/100
16/16 [==============================] - 4s 223ms/step - loss: 5.7544 - accuracy: 0.0137 - val_loss: 5.7683 - val_accuracy: 0.0255
Epoch 4/100
16/16 [==============================] - 3s 217ms/step - loss: 5.5575 - accuracy: 0.0183 - val_loss: 5.6165 - val_accuracy: 0.0064
Epoch 5/100
16/16 [==============================] - 3s 216ms/step - loss: 5.2358 - accuracy: 0.0281 - val_loss: 5.2688 - val_accuracy: 0.0510
Epoch 6/100
16/16 [==============================] - 4s 220ms/step - loss: 4.7533 - accuracy: 0.0490 - val_loss: 4.9237 - val_accuracy: 0.0446
Epoch 7/100
16/16 [==============================] - 3s 217ms/step - loss: 4.4150 - accuracy: 0.0791 - val_loss: 4.7015 - val_accuracy: 0.0764
Epoch 8/100

In [ ]:
print('Chatbot started. type /stop to stop the bot')
def runChatbot():
	while(True):
		sen=input('QUERY : ')
		if sen=='/stop':
			break
		tokens = Tokenizer.texts_to_sequences([sen])
		tokens = pad_sequences(tokens, maxlen = 64)
		prediction = model.predict(np.array(tokens))
		pred = np.argmax(prediction)
		print('RESPONSE : ',responses[classes[pred]])
runChatbot()

Chatbot started. type /stop to stop the bot
QUERY : tell about isro
RESPONSE :  ISRO is the primary agency in India to perform tasks related to space based applications, space exploration and development of related technologies
QUERY : tell me about pslv
RESPONSE :  Polar Satellite Launch Vehicle
QUERY : what is gslv mark 3
RESPONSE :   Geosynchronous Satellite Launch Vehicle Mark II GSLV Mk II is the largest launch vehicle developed by India, which is currently in operation
QUERY : /stop
